In [63]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, LabelEncoder
from scipy.stats import zscore

In [39]:
#@title preprocessing_classes
# class Duplicates:
#     def __init__(self, duplicates=True):
#         self.duplicates = duplicates

#     def handle(self, df):
#         if self.duplicates:
#             df.drop_duplicates(inplace=True, ignore_index=True)
#         return df

# class MissingValues:
#     def __init__(self, missing_num=None, missing_categ=None):
#         self.missing_num = missing_num
#         self.missing_categ = missing_categ

#     def handle(self, df, _n_neighbors=5):
#         if self.missing_num or self.missing_categ:
#             if df.isna().sum().sum() != 0:
#                 if self.missing_num:
#                     df = self._handle_missing_num(df, _n_neighbors)
#                 if self.missing_categ:
#                     df = self._handle_missing_categ(df, _n_neighbors)
#         return df

#     def _handle_missing_num(self, df, _n_neighbors):
#         num_cols = df.select_dtypes(include=np.number).columns
#         for col in num_cols:
#             # if self.missing_num == 'delete':
#             #     df = df.dropna(subset=[col])
#             if self.missing_num in ['auto', 'knn']:  # Use KNN imputation
#                 imputer = KNNImputer(n_neighbors=_n_neighbors)
#                 df[col] = imputer.fit_transform(df[[col]])
#                 df[col] = df[col].round().astype('Int64')
#         return df

#     def _handle_missing_categ(self, df, _n_neighbors):
#         cat_cols = set(df.columns) - set(df.select_dtypes(include=np.number).columns)
#         for col in cat_cols:
#             # if self.missing_categ == 'delete':
#             #     df = df.dropna(subset=[col])
#             if self.missing_categ in ['auto', 'logreg', 'most_frequent']:
#                 if self.missing_categ == 'most_frequent':
#                     strategy = self.missing_categ
#                 else:
#                     strategy = 'constant'
#                 imputer = SimpleImputer(strategy=strategy)
#                 df[col] = imputer.fit_transform(df[[col]])
#         return df

# # class Outliers:
# #     def __init__(self, method=None, threshold=3):
# #         self.method = method
# #         self.threshold = threshold

# #     def handle(self, df):
# #         if self.method:
# #             if self.method == 'zscore':
# #                 df = self._handle_zscore(df)
# #             elif self.method == 'iqr':
# #                 df = self._handle_iqr(df)
# #         return df

# #     def _handle_zscore(self, df):
# #         num_cols = df.select_dtypes(include=np.number).columns
# #         for col in num_cols:
# #             z_scores = np.abs(zscore(df[col]))
# #             df = df[(z_scores < self.threshold).all(axis=1)]
# #         return df

# #     def _handle_iqr(self, df):
# #         num_cols = df.select_dtypes(include=np.number).columns
# #         for col in num_cols:
# #             q1 = df[col].quantile(0.25)
# #             q3 = df[col].quantile(0.75)
# #             iqr = q3 - q1
# #             lower_bound = q1 - (self.threshold * iqr)
# #             upper_bound = q3 + (self.threshold * iqr)
# #             df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
# #         return df
# class Outliers:
#     def __init__(self, method=None, threshold=3):
#         self.method = method
#         self.threshold = threshold

#     def handle(self, df):
#         if self.method == 'knn':  # Use KNN for imputation
#             df = self._handle_knn(df)
#         return df

#     def _handle_knn(self, df):
#         imputer = KNNImputer()
#         df[df.columns] = imputer.fit_transform(df)
#         return df


# class Adjust:
#     def __init__(self, scaler=None, extract_datetime=False):
#         self.scaler = scaler
#         self.extract_datetime = extract_datetime

#     def handle(self, df):
#         if self.scaler:
#             if self.scaler in ['minmax', 'standard', 'robust']:
#                 scaler = preprocessing.__getattribute__(self.scaler.capitalize()+'Scaler')()
#                 df[df.columns] = scaler.fit_transform(df[df.columns])
#         if self.extract_datetime:
#             df = self._convert_datetime(df)
#         return df

#     def _convert_datetime(self, df):
#         cols = set(df.columns) ^ set(df.select_dtypes(include=np.number).columns)
#         for col in cols:
#             try:
#                 df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
#                 if self.extract_datetime != False:
#                     df = df.join(pd.to_datetime(df[col]).dt.__getattribute__(self.extract_datetime))
#             except:
#                 pass
#         return df

# class EncodeCateg:
#     def __init__(self, encode_categ=None):
#         self.encode_categ = encode_categ

#     def handle(self, df):
#         if self.encode_categ:
#             if self.encode_categ == 'auto':
#                 self._auto_encode(df)
#             elif isinstance(self.encode_categ, list):
#                 for col in self.encode_categ:
#                     if col in df.columns:
#                         self._auto_encode(df, col)
#         return df

#     def _auto_encode(self, df, col=None):
#         if col:
#             if len(df[col].unique()) <= 10:
#                 df = pd.get_dummies(df, columns=[col], prefix=[col])
#             else:
#                 le = LabelEncoder()
#                 df[col] = le.fit_transform(df[col])
#         else:
#             for col in df.select_dtypes(include='object'):
#                 if len(df[col].unique()) <= 10:
#                     df = pd.get_dummies(df, columns=[col], prefix=[col])
#                 else:
#                     le = LabelEncoder()
#                     df[col] = le.fit_transform(df[col])
#         return df



In [40]:
#@title master_class
# class AutoDataCleaner:
#     def __init__(self, duplicates=True, missing_strategy=None, outliers_method=None,
#                  threshold=3, scaling_method=None, extract_datetime=False, encode_categorical=None):
#         self.duplicates = duplicates
#         self.missing_strategy = missing_strategy
#         self.outliers_method = outliers_method
#         self.threshold = threshold
#         self.scaling_method = scaling_method
#         self.extract_datetime = extract_datetime
#         self.encode_categorical = encode_categorical

#     def fit_transform(self, df):
#         if self.duplicates:
#             df = self.remove_duplicates(df)
#         if self.missing_strategy:
#             df = self.handle_missing_values(df)
#         if self.outliers_method:
#             df = self.handle_outliers(df)
#         if self.scaling_method:
#             df = self.adjust_scaling(df)
#         if self.extract_datetime:
#             df = self.convert_datetime(df)
#         if self.encode_categorical:
#             df = self.encode_categorical_features(df)
#         return df

#     def remove_duplicates(self, df):
#         df.drop_duplicates(inplace=True, ignore_index=True)
#         return df

#     def handle_missing_values(self, df):
#         num_cols = df.select_dtypes(include=np.number).columns
#         cat_cols = df.select_dtypes(include='object').columns

#         # if 'delete' in self.missing_strategy:
#         #     df.dropna(subset=num_cols, inplace=True)
#         #     df.dropna(subset=cat_cols, inplace=True)
#         if 'auto' in self.missing_strategy:
#             df[num_cols] = self.impute_missing_values(df[num_cols], strategy='numeric')
#             df[cat_cols] = self.impute_missing_values(df[cat_cols], strategy='categorical')

#         return df

#     def impute_missing_values(self, df, strategy='numeric'):
#         imputer = None
#         if 'knn' in self.missing_strategy:
#             if strategy == 'numeric':
#                 imputer = KNeighborsRegressor()
#             elif strategy == 'categorical':
#                 imputer = KNeighborsClassifier()
#         elif 'most_frequent' in self.missing_strategy:
#             imputer = SimpleImputer(strategy='most_frequent')

#         if imputer:
#             df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

#         return df

#     def handle_outliers(self, df):
#         num_cols = df.select_dtypes(include=np.number).columns

#         if 'zscore' in self.outliers_method:
#             df = self.remove_zscore_outliers(df[num_cols])
#         elif 'iqr' in self.outliers_method:
#             df = self.remove_iqr_outliers(df[num_cols])

#         return df

#     def remove_zscore_outliers(self, df):
#         z_scores = np.abs(zscore(df))
#         df = df[(z_scores < self.threshold).all(axis=1)]
#         return df

#     def remove_iqr_outliers(self, df):
#         for col in df.columns:
#             q1 = df[col].quantile(0.25)
#             q3 = df[col].quantile(0.75)
#             iqr = q3 - q1
#             lower_bound = q1 - (self.threshold * iqr)
#             upper_bound = q3 + (self.threshold * iqr)
#             df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
#         return df

#     def adjust_scaling(self, df):
#         scaler = None
#         if 'minmax' in self.scaling_method:
#             scaler = MinMaxScaler()
#         elif 'standard' in self.scaling_method:
#             scaler = StandardScaler()
#         elif 'robust' in self.scaling_method:
#             scaler = RobustScaler()

#         if scaler:
#             df[df.columns] = scaler.fit_transform(df[df.columns])
#         return df

#     def convert_datetime(self, df):
#         cols = df.select_dtypes(include='datetime64').columns
#         for col in cols:
#             df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
#             if self.extract_datetime != False:
#                 df = df.join(pd.to_datetime(df[col]).dt.__getattribute__(self.extract_datetime))

#         return df

#     def encode_categorical_features(self, df):
#         if 'auto' in self.encode_categorical:
#             for col in df.select_dtypes(include='object'):
#                 if len(df[col].unique()) <= 10:
#                     df = pd.get_dummies(df, columns=[col], prefix=[col])
#                 else:
#                     le = LabelEncoder()
#                     df[col] = le.fit_transform(df[col])
#         return df


In [46]:
#@title preprocessing_classes2
class Duplicates:
    def __init__(self, duplicates=True):
        self.duplicates = duplicates

    def handle(self, df):
        if self.duplicates:
            df.drop_duplicates(inplace=True, ignore_index=True)
        return df

class MissingValues:
    def __init__(self, missing_num=None, missing_categ=None):
        self.missing_num = missing_num
        self.missing_categ = missing_categ

    def handle(self, df, _n_neighbors=5):
        if self.missing_num or self.missing_categ:
            if df.isna().sum().sum() != 0:
                if self.missing_num:
                    df = self._handle_missing_num(df, _n_neighbors)
                if self.missing_categ:
                    df = self._handle_missing_categ(df, _n_neighbors)
        return df

    def _handle_missing_num(self, df, _n_neighbors):
        num_cols = df.select_dtypes(include=np.number).columns
        for col in num_cols:
            if self.missing_num in ['auto', 'knn']:  # Use KNN imputation
                imputer = KNNImputer(n_neighbors=_n_neighbors)
                df[col] = imputer.fit_transform(df[[col]])
                df[col] = df[col].round().astype('Int64')
        return df

    def _handle_missing_categ(self, df, _n_neighbors):
        cat_cols = set(df.columns) - set(df.select_dtypes(include=np.number).columns)
        for col in cat_cols:
            if self.missing_categ in ['auto', 'logreg', 'most_frequent']:
                if self.missing_categ == 'most_frequent':
                    strategy = self.missing_categ
                else:
                    strategy = 'constant'
                imputer = SimpleImputer(strategy=strategy)
                df[col] = imputer.fit_transform(df[[col]])
        return df

class Outliers:
    def __init__(self, method=None, threshold=3):
        self.method = method
        self.threshold = threshold

    def handle(self, df):
        if self.method == 'knn':  # Use KNN for imputation
            df = self._handle_knn(df)
        return df

    def _handle_knn(self, df):
        imputer = KNNImputer()
        df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

        # Identify outliers using z-score
        z_scores = np.abs(zscore(df_imputed))
        outlier_indices = np.any(z_scores > self.threshold, axis=1)

        # If outliers are detected, apply a different imputation strategy
        if np.any(outlier_indices):
            df_outliers = df_imputed[outlier_indices]
            df_outliers.fillna(df_outliers.mean(), inplace=True)  # Replace NaNs with mean
            df_imputed.loc[outlier_indices] = df_outliers

        return df_imputed


class Adjust:
    def __init__(self, scaler=None, extract_datetime=False):
        self.scaler = scaler
        self.extract_datetime = extract_datetime

    def handle(self, df):
        if self.scaler:
            if self.scaler in ['minmax', 'standard', 'robust']:
                scaler = preprocessing.__getattribute__(self.scaler.capitalize()+'Scaler')()
                df[df.columns] = scaler.fit_transform(df[df.columns])
        if self.extract_datetime:
            df = self._convert_datetime(df)
        return df

    def _convert_datetime(self, df):
        cols = set(df.columns) ^ set(df.select_dtypes(include=np.number).columns)
        for col in cols:
            try:
                df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
                if self.extract_datetime != False:
                    df = df.join(pd.to_datetime(df[col]).dt.__getattribute__(self.extract_datetime))
            except:
                pass
        return df

class EncodeCateg:
    def __init__(self, encode_categ=None):
        self.encode_categ = encode_categ

    def handle(self, df):
        if self.encode_categ:
            if self.encode_categ == 'auto':
                self._auto_encode(df)
            elif isinstance(self.encode_categ, list):
                for col in self.encode_categ:
                    if col in df.columns:
                        self._auto_encode(df, col)
        return df

    def _auto_encode(self, df, col=None):
        if col:
            if len(df[col].unique()) <= 10:
                df = pd.get_dummies(df, columns=[col], prefix=[col])
            else:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col])
        else:
            for col in df.select_dtypes(include='object'):
                if len(df[col].unique()) <= 10:
                    df = pd.get_dummies(df, columns=[col], prefix=[col])
                else:
                    le = LabelEncoder()
                    df[col] = le.fit_transform(df[col])
        return df


In [47]:
#@title master_class2
class AutoDataCleaner:
    def __init__(self, duplicates=True, missing_strategy=None, outliers_method=None,
                 threshold=3, scaling_method=None, extract_datetime=False, encode_categorical=None):
        self.duplicates = duplicates
        self.missing_strategy = missing_strategy
        self.outliers_method = outliers_method
        self.threshold = threshold
        self.scaling_method = scaling_method
        self.extract_datetime = extract_datetime
        self.encode_categorical = encode_categorical

    def fit_transform(self, df):
        if self.duplicates:
            df = self.remove_duplicates(df)
        if self.missing_strategy:
            df = self.handle_missing_values(df)
        if self.outliers_method:
            df = self.handle_outliers(df)
        if self.scaling_method:
            df = self.adjust_scaling(df)
        if self.extract_datetime:
            df = self.convert_datetime(df)
        if self.encode_categorical:
            df = self.encode_categorical_features(df)
        return df

    def remove_duplicates(self, df):
        df.drop_duplicates(inplace=True, ignore_index=True)
        return df

    def handle_missing_values(self, df):
        num_cols = df.select_dtypes(include=np.number).columns
        cat_cols = df.select_dtypes(include='object').columns

        if 'auto' in self.missing_strategy:
            df[num_cols] = self.impute_missing_values(df[num_cols], strategy='numeric')
            df[cat_cols] = self.impute_missing_values(df[cat_cols], strategy='categorical')

        return df

    def impute_missing_values(self, df, strategy='numeric'):
        imputer = None
        if 'knn' in self.missing_strategy:
            if strategy == 'numeric':
                imputer = KNeighborsRegressor()
            elif strategy == 'categorical':
                imputer = KNeighborsClassifier()
        elif 'most_frequent' in self.missing_strategy:
            imputer = SimpleImputer(strategy='most_frequent')

        if imputer:
            df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

        return df

    def handle_outliers(self, df):
        num_cols = df.select_dtypes(include=np.number).columns

        if 'knn' in self.outliers_method:
            df = self.handle_outliers_knn(df[num_cols])
        else:
            df = self.remove_outliers(df[num_cols])

        return df

    def handle_outliers_knn(self, df):
        imputer = KNNImputer()
        df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

        z_scores = np.abs(zscore(df_imputed))
        outlier_indices = np.any(z_scores > self.threshold, axis=1)

        if np.any(outlier_indices):
            df_outliers = df_imputed[outlier_indices]
            df_outliers.fillna(df_outliers.mean(), inplace=True)
            df_imputed.loc[outlier_indices] = df_outliers

        return df_imputed

    def remove_outliers(self, df):
        if 'zscore' in self.outliers_method:
            z_scores = np.abs(zscore(df))
            df = df[(z_scores < self.threshold).all(axis=1)]
        elif 'iqr' in self.outliers_method:
            for col in df.columns:
                q1 = df[col].quantile(0.25)
                q3 = df[col].quantile(0.75)
                iqr = q3 - q1
                lower_bound = q1 - (self.threshold * iqr)
                upper_bound = q3 + (self.threshold * iqr)
                df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
        return df

    def adjust_scaling(self, df):
        scaler = None
        if 'minmax' in self.scaling_method:
            scaler = MinMaxScaler()
        elif 'standard' in self.scaling_method:
            scaler = StandardScaler()
        elif 'robust' in self.scaling_method:
            scaler = RobustScaler()

        if scaler:
            df[df.columns] = scaler.fit_transform(df[df.columns])
        return df

    def convert_datetime(self, df):
        cols = df.select_dtypes(include='datetime64').columns
        for col in cols:
            df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
            if self.extract_datetime != False:
                df = df.join(pd.to_datetime(df[col]).dt.__getattribute__(self.extract_datetime))

        return df

    def encode_categorical_features(self, df):
        if 'auto' in self.encode_categorical:
            for col in df.select_dtypes(include='object'):
                if len(df[col].unique()) <= 10:
                    df = pd.get_dummies(df, columns=[col], prefix=[col])
                else:
                    le = LabelEncoder()
                    df[col] = le.fit_transform(df[col])
        return df



In [48]:
# Load CSV data into a DataFrame
df = pd.read_csv("kidney_disease.csv")

# Initialize AutoDataCleaner with desired configurations
cleaner = AutoDataCleaner(duplicates=True,
                           missing_strategy=['auto', 'most_frequent'],
                           outliers_method=['zscore', 'iqr'],
                           threshold=3,
                           scaling_method=['minmax', 'standard'],
                           extract_datetime=True,
                           encode_categorical='auto')

# Apply data cleaning
cleaned_df = cleaner.fit_transform(df)

In [50]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    float64
 1   age             400 non-null    float64
 2   bp              400 non-null    float64
 3   sg              400 non-null    float64
 4   al              400 non-null    float64
 5   su              400 non-null    float64
 6   rbc             400 non-null    object 
 7   pc              400 non-null    object 
 8   pcc             400 non-null    object 
 9   ba              400 non-null    object 
 10  bgr             400 non-null    float64
 11  bu              400 non-null    float64
 12  sc              400 non-null    float64
 13  sod             400 non-null    float64
 14  pot             400 non-null    float64
 15  hemo            400 non-null    float64
 16  pcv             400 non-null    object 
 17  wc              400 non-null    obj

In [51]:
print(cleaned_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361 entries, 0 to 399
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      361 non-null    float64
 1   age     361 non-null    float64
 2   bp      361 non-null    float64
 3   sg      361 non-null    float64
 4   al      361 non-null    float64
 5   su      361 non-null    float64
 6   bgr     361 non-null    float64
 7   bu      361 non-null    float64
 8   sc      361 non-null    float64
 9   sod     361 non-null    float64
 10  pot     361 non-null    float64
 11  hemo    361 non-null    float64
dtypes: float64(12)
memory usage: 36.7 KB
None


In [ ]:
##########################################################################################
#@title master_class1

class AutoDataCleaner:
    def __init__(self, duplicates=True, missing_strategy=None, outliers_method=None,
                 threshold=3, scaling_method=None, extract_datetime=False, encode_categorical=None):
        self.duplicates = duplicates
        self.missing_strategy = missing_strategy
        self.outliers_method = outliers_method
        self.threshold = threshold
        self.scaling_method = scaling_method
        self.extract_datetime = extract_datetime
        self.encode_categorical = encode_categorical

    def fit_transform(self, df):
        if self.duplicates:
            df = self.remove_duplicates(df)
        if self.missing_strategy:
            df = self.handle_missing_values(df)
        if self.outliers_method:
            df = self.handle_outliers(df)
        if self.scaling_method:
            df = self.adjust_scaling(df)
        if self.extract_datetime:
            df = self.convert_datetime(df)
        if self.encode_categorical:
            df = self.encode_categorical_features(df)
        return df

    def remove_duplicates(self, df):
        df.drop_duplicates(inplace=True, ignore_index=True)
        return df

    def handle_missing_values(self, df):
        if self.missing_strategy == 'delete':
            df.dropna(inplace=True)
        elif self.missing_strategy == 'auto':
            df = self.impute_missing_values(df)
        return df

    def impute_missing_values(self, df):
        imputer = None
        if 'knn' in self.missing_strategy:
            imputer = KNNImputer()
        elif 'most_frequent' in self.missing_strategy:
            imputer = SimpleImputer(strategy='most_frequent')

        if imputer:
            df[df.columns] = imputer.fit_transform(df)
        return df

    def handle_outliers(self, df):
        if self.outliers_method == 'zscore':
            df = self.remove_zscore_outliers(df)
        elif self.outliers_method == 'iqr':
            df = self.remove_iqr_outliers(df)
        return df

    def remove_zscore_outliers(self, df):
        z_scores = np.abs(zscore(df))
        df = df[(z_scores < self.threshold).all(axis=1)]
        return df

    def remove_iqr_outliers(self, df):
        num_cols = df.select_dtypes(include=np.number).columns
        for col in num_cols:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - (self.threshold * iqr)
            upper_bound = q3 + (self.threshold * iqr)
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
        return df

    def adjust_scaling(self, df):
        scaler = None
        if self.scaling_method == 'minmax':
            scaler = MinMaxScaler()
        elif self.scaling_method == 'standard':
            scaler = StandardScaler()
        elif self.scaling_method == 'robust':
            scaler = RobustScaler()

        if scaler:
            df[df.columns] = scaler.fit_transform(df[df.columns])
        return df

    def convert_datetime(self, df):
        cols = set(df.columns) ^ set(df.select_dtypes(include=np.number).columns)
        for col in cols:
            try:
                df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
                if self.extract_datetime != False:
                    df = df.join(pd.to_datetime(df[col]).dt.__getattribute__(self.extract_datetime))
            except:
                pass
        return df

    def encode_categorical_features(self, df):
        if self.encode_categorical == 'auto':
            for col in df.select_dtypes(include='object'):
                if len(df[col].unique()) <= 10:
                    df = pd.get_dummies(df, columns=[col], prefix=[col])
                else:
                    le = LabelEncoder()
                    df[col] = le.fit_transform(df[col])
        return df

In [32]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(21)

# Generate synthetic dataset
num_samples = 1000
num_features = 10

# Generate random data for each feature
data = {
    f'Feature_{i}': np.random.randn(num_samples) for i in range(1, num_features + 1)
}

# Introduce missing values
missing_percentage = 0.1
for feature in data.keys():
    missing_indices = np.random.choice(num_samples, int(missing_percentage * num_samples), replace=False)
    data[feature][missing_indices] = np.nan

# Introduce outliers
outlier_percentage = 0.05
for feature in data.keys():
    outlier_indices = np.random.choice(num_samples, int(outlier_percentage * num_samples), replace=False)
    data[feature][outlier_indices] = np.random.uniform(low=-10, high=10, size=len(outlier_indices))

# Create DataFrame
df = pd.DataFrame(data)

# Print some basic statistics
print("Basic statistics before preprocessing:")
print(df.describe())

# Save the dataset to a CSV file
df.to_csv('synthetic_dataset2.csv', index=False)


Basic statistics before preprocessing:
        Feature_1   Feature_2   Feature_3   Feature_4   Feature_5   Feature_6  \
count  902.000000  906.000000  905.000000  906.000000  901.000000  905.000000   
mean    -0.072030    0.005005    0.011028    0.098298   -0.013162   -0.059704   
std      1.651788    1.652416    1.759215    1.628810    1.740880    1.768028   
min     -9.842061   -9.281973   -9.539031   -9.952659   -9.900852   -9.973520   
25%     -0.715977   -0.709270   -0.694639   -0.663724   -0.728232   -0.701059   
50%      0.003219    0.009753    0.053530    0.011458    0.031356   -0.035066   
75%      0.689010    0.718652    0.740827    0.713994    0.777602    0.715739   
max      9.871996    9.862402    9.998826    9.888158    8.374322    9.135956   

        Feature_7   Feature_8   Feature_9  Feature_10  
count  905.000000  905.000000  903.000000  908.000000  
mean     0.108368   -0.008339    0.048426    0.124866  
std      1.679392    1.822530    1.715898    1.625668  
min    